# Widgets and Layout Customization

As of version 0.5 DataMapPlot supports a flexible widget system that allows you to add and arrange interactive UI components in your data map visualizations. Widgets can be placed in any of the four corners of the plot, or in slide-out drawers on the left or right side. This notebook will walk you through the available widgets, how to configure their placement, and how to make use of drawers to create cleaner layouts with tools accessible on demand.

To get started we'll need to import DataMapPlot.

In [ ]:
import datamapplot
import numpy as np
import requests
import io

## Loading Sample Data

First, let's load some data to work with. We'll use the ArXiv ML data map from the DataMapPlot repository.

In [ ]:
base_url = "https://github.com/TutteInstitute/datamapplot/raw/main/examples"

data_map_file = requests.get(f"{base_url}/arxiv_ml_data_map.npz")
arxivml_data_map = np.load(io.BytesIO(data_map_file.content))["arr_0"]

arxivml_label_layers = []
for layer_num in range(5):
    label_file = requests.get(f"{base_url}/arxiv_ml_layer{layer_num}_cluster_labels.npz")
    arxivml_label_layers.append(
        np.load(io.BytesIO(label_file.content), allow_pickle=True)["arr_0"]
    )

hover_data_file = requests.get(f"{base_url}/arxiv_ml_hover_data.npz")
arxiv_hover_data = np.load(io.BytesIO(hover_data_file.content), allow_pickle=True)["arr_0"]

## Available Widgets

DataMapPlot provides several built-in widgets:

| Widget | Description | Parameters |
|--------|-------------|------------|
| `TitleWidget` | Displays a title and optional subtitle | `title`, `sub_title`, `title_font_size`, `sub_title_font_size`, `title_font_color`, `title_font_family` |
| `SearchWidget` | Text search functionality for hover data | `search_field` (default: "hover_text") |
| `LogoWidget` | Displays a logo image | `logo` (URL or path), `logo_width` |
| `LegendWidget` | Color legend for points | `legend_title` |
| `HistogramWidget` | Time-based histograms with selection | `histogram_data`, `histogram_date_format`, `histogram_n_bins` |
| `ColormapSelectorWidget` | Switch between colormaps | `available_colormaps` |
| `TopicTreeWidget` | Hierarchical topic browser | Uses label layers automatically |

Let's look at some examples of how to use these widgets.

## Basic Widget Usage

The simplest way to use widgets is through the legacy parameters that are automatically converted to widgets. For example, `enable_search=True` creates a `SearchWidget`, and providing `title` and `sub_title` creates a `TitleWidget`.

In [ ]:
# Using legacy parameters (automatically converted to widgets)
plot = datamapplot.create_interactive_plot(
    arxivml_data_map,
    arxivml_label_layers[0],
    arxivml_label_layers[2],
    arxivml_label_layers[4],
    hover_text=arxiv_hover_data,
    font_family="Playfair Display SC",
    title="ArXiv ML Landscape",
    sub_title="A data map of papers from the Machine Learning section of ArXiv",
    logo="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/ArXiv_logo_2022.svg/512px-ArXiv_logo_2022.svg.png",
    logo_width=180,
    enable_search=True,
    darkmode=True,
)
plot

## Using Widget Classes for Custom Placement

For more control over widget placement, you can directly instantiate widget classes. Each widget class accepts a `location` parameter to specify where it should appear.

### Widget Locations

Widgets can be placed in any of these locations:

- **Corners**: `"top-left"`, `"top-right"`, `"bottom-left"`, `"bottom-right"`
- **Drawers**: `"drawer-left"`, `"drawer-right"`

### Widget Stacking

Multiple widgets in the same location are stacked. You can control the stacking order with the `order` parameter - lower numbers appear first (top for corners, first in drawer).

In [ ]:
from datamapplot.widgets import TitleWidget, SearchWidget, LogoWidget, TopicTreeWidget

# Create widgets with custom placement by instantiating widget classes
widgets = [
    TitleWidget(
        title="ArXiv ML Landscape",
        sub_title="Papers from the ML section",
        title_font_size=32,
        location="top-left",
        order=0,
        darkmode=True,  # Match the plot's darkmode setting
    ),
    SearchWidget(
        search_field="hover_text",
        location="top-right",
        order=0,
    ),
    LogoWidget(
        logo="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/ArXiv_logo_2022.svg/512px-ArXiv_logo_2022.svg.png",
        logo_width=128,
        location="bottom-right",
        order=0,
    ),
]

plot = datamapplot.create_interactive_plot(
    arxivml_data_map,
    arxivml_label_layers[0],
    arxivml_label_layers[2],
    arxivml_label_layers[4],
    hover_text=arxiv_hover_data,
    font_family="Playfair Display SC",
    darkmode=True,
    widgets=widgets,
)
plot

## Slide-Out Drawers

For more complex UIs or when you want to keep the main view clean, you can place widgets in slide-out drawers. Drawers are hidden by default and can be opened by clicking a handle on the edge of the screen.

**Keyboard shortcuts:**
- `Ctrl + [` - Toggle left drawer
- `Ctrl + ]` - Toggle right drawer
- `Escape` - Close all drawers

Let's put the search and a topic tree in a left drawer:

In [ ]:
widgets = [
    # Title in corner
    TitleWidget(
        title="ArXiv ML Landscape",
        sub_title="Click the left edge to explore topics",
        location="top-left",
        order=0,
        darkmode=True,
    ),
    # Search in left drawer
    SearchWidget(
        search_field="hover_text",
        location="drawer-left",
        order=0,
    ),
    # Topic tree in left drawer (after search)
    TopicTreeWidget(
        location="drawer-left",
        order=1,
    ),
    # Logo in corner
    LogoWidget(
        logo="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/ArXiv_logo_2022.svg/512px-ArXiv_logo_2022.svg.png",
        logo_width=128,
        location="bottom-right",
        order=0,
    ),
]

plot = datamapplot.create_interactive_plot(
    arxivml_data_map,
    arxivml_label_layers[0],
    arxivml_label_layers[2],
    arxivml_label_layers[4],
    hover_text=arxiv_hover_data,
    font_family="Playfair Display SC",
    darkmode=True,
    widgets=widgets,
)
plot

## Using JSON Configuration Files for Layout Presets

For reusable layouts, you can use JSON configuration files to override widget placement. These files specify where widgets created by legacy parameters should be placed. DataMapPlot includes several built-in preset configurations:

- `minimal` - Just title and logo
- `default` - Title, logo, and search  
- `analyst` - Search and topic tree in left drawer, legend and colormap selector in right drawer
- `presentation` - Clean layout for presenting

In [ ]:
# Using a preset configuration name
plot = datamapplot.create_interactive_plot(
    arxivml_data_map,
    arxivml_label_layers[0],
    arxivml_label_layers[2],
    arxivml_label_layers[4],
    hover_text=arxiv_hover_data,
    font_family="Playfair Display SC",
    title="ArXiv ML Landscape",
    sub_title="Using 'analyst' widget layout",
    logo="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/ArXiv_logo_2022.svg/512px-ArXiv_logo_2022.svg.png",
    darkmode=True,
    default_widget_config="analyst",
)
plot

## Creating a Custom JSON Layout Configuration

You can create your own JSON configuration file to define widget placements. The format uses widget IDs as keys and specifies location, order, visibility, and collapsibility:

In [ ]:
import json

# Example custom layout configuration
# Widget IDs are keys, with location/order/visible as properties
custom_layout = {
    "_description": "A custom layout for data exploration",
    "title": {
        "location": "top-left",
        "order": 0,
        "visible": True
    },
    "search": {
        "location": "drawer-left",
        "order": 0,
        "visible": True
    },
    "topic-tree": {
        "location": "drawer-left",
        "order": 1,
        "visible": True,
        "collapsible": False
    },
    "logo": {
        "location": "bottom-right",
        "order": 0,
        "visible": True
    }
}

print(json.dumps(custom_layout, indent=2))

## Mixing Legacy Parameters with Widgets

You can mix legacy parameters with explicit widgets. Legacy parameters are converted to widgets automatically and merged with any explicit widgets you provide. If you provide an explicit widget of the same type, it takes precedence.

In [ ]:
# The title comes from legacy parameter
# The search widget is explicitly configured to be in a drawer
widgets = [
    SearchWidget(
        location="drawer-left",
        order=0,
    ),
]

plot = datamapplot.create_interactive_plot(
    arxivml_data_map,
    arxivml_label_layers[0],
    arxivml_label_layers[2],
    arxivml_label_layers[4],
    hover_text=arxiv_hover_data,
    font_family="Playfair Display SC",
    title="ArXiv ML Landscape",
    sub_title="Search is in the left drawer",
    logo="https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/ArXiv_logo_2022.svg/512px-ArXiv_logo_2022.svg.png",
    logo_width=128,
    enable_search=True,  # This is overridden by the explicit widget above
    darkmode=True,
    widgets=widgets,
)
plot

## Widget Options Reference

### TitleWidget Options
- `title` (str): Main title text
- `sub_title` (str): Subtitle text
- `title_font_size` (str): CSS font size, e.g., "2rem"
- `sub_title_font_size` (str): CSS font size for subtitle
- `title_font_color` (str): CSS color value
- `title_font_family` (str): Font family name

### SearchWidget Options
- `search_field` (str): Field name to search (default: "hover_text")

### LogoWidget Options
- `logo` (str): URL or file path to logo image
- `logo_width` (int): Width in pixels

### HistogramWidget Options
- `histogram_data` (array): Date/time data for histogram
- `histogram_date_format` (str): Date format string
- `histogram_n_bins` (int): Number of bins
- `histogram_group_by` (str): Field to group by
- `histogram_width` (str): CSS width

### LegendWidget Options
- `legend_title` (str): Title for the legend

### ColormapSelectorWidget Options
- `available_colormaps` (list): List of colormap names to offer

### TopicTreeWidget Options
- Uses label layers automatically, no special options required

## Advanced: Creating Custom Widgets

You can create custom widgets by subclassing `WidgetBase`. A widget needs to provide:

- `html` property: HTML markup for the widget
- `css` property: CSS styles (optional)
- `javascript` property: JavaScript code (optional)
- `dependencies` property: List of external dependencies (optional)

```python
from datamapplot.widgets import WidgetBase

class MyCustomWidget(WidgetBase):
    def __init__(self, message="Hello"):
        super().__init__()
        self.message = message
    
    @property
    def html(self):
        return f'<div class="my-widget">{self.message}</div>'
    
    @property
    def css(self):
        return '.my-widget { padding: 10px; background: #333; color: #fff; }'
    
    @property
    def javascript(self):
        return 'console.log("MyCustomWidget initialized");'
```

## Summary

The widget system in DataMapPlot provides:

1. **Backward compatibility**: Legacy parameters continue to work
2. **Flexible placement**: Choose from 4 corners or 2 drawers
3. **Easy configuration**: Use `WidgetConfig` or JSON files
4. **Preset layouts**: Built-in configurations for common use cases
5. **Extensibility**: Create custom widgets by subclassing `WidgetBase`

For more examples, check out the examples gallery which includes demonstrations of different widget configurations.